In [49]:
import os

In [50]:
%pwd

'/Users/saee2510/Desktop/projects/kidney-disease-classification-dl/Kidney-Disease-Classification-Deep-Learning-Project'

In [51]:
os.chdir("../")

In [52]:
%pwd

'/Users/saee2510/Desktop/projects/kidney-disease-classification-dl'

In [53]:
import dagshub
dagshub.init(repo_owner='saeee2510', repo_name='Kidney-Disease-Classification-Deep-Learning-Project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "saeee2510/Kidney-Disease-Classification-Deep-Learning-Project"

[2024-06-28 13:31:33,472: INFO: helpers: Initialized MLflow to track repo "saeee2510/Kidney-Disease-Classification-Deep-Learning-Project"]


Repository saeee2510/Kidney-Disease-Classification-Deep-Learning-Project initialized!

[2024-06-28 13:31:33,475: INFO: helpers: Repository saeee2510/Kidney-Disease-Classification-Deep-Learning-Project initialized!]


In [54]:
import os
import dagshub

# Initialize DagsHub integration
dagshub.init(repo_owner='saeee2510', repo_name='Kidney-Disease-Classification-Deep-Learning-Project', mlflow=True)

# Print the environment variables to verify
print(os.environ.get("MLFLOW_TRACKING_URI"))
print(os.environ.get("MLFLOW_TRACKING_USERNAME"))
print(os.environ.get("MLFLOW_TRACKING_PASSWORD"))

import mlflow

with mlflow.start_run():
    mlflow.log_param('parameter name', 'value')
    mlflow.log_metric('metric name', 1)


Initialized MLflow to track repo "saeee2510/Kidney-Disease-Classification-Deep-Learning-Project"

[2024-06-28 13:31:40,601: INFO: helpers: Initialized MLflow to track repo "saeee2510/Kidney-Disease-Classification-Deep-Learning-Project"]


Repository saeee2510/Kidney-Disease-Classification-Deep-Learning-Project initialized!

[2024-06-28 13:31:40,603: INFO: helpers: Repository saeee2510/Kidney-Disease-Classification-Deep-Learning-Project initialized!]
https://dagshub.com/saeee2510/Kidney-Disease-Classification-Deep-Learning-Project.mlflow
80888df6093c02b9a413a170517c38d919ab9e71
80888df6093c02b9a413a170517c38d919ab9e71


In [55]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/saeee2510/Kidney-Disease-Classification-Deep-Learning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="80888df6093c02b9a413a170517c38d919ab9e71"
os.environ["MLFLOW_TRACKING_PASSWORD"]="80888df6093c02b9a413a170517c38d919ab9e71"

In [56]:
import tensorflow as tf

In [58]:
model = tf.keras.models.load_model("Kidney-Disease-Classification-Deep-Learning-Project/artifacts/training/model.h5")

In [59]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [60]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [61]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/saeee2510/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [62]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [67]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [68]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config/config.yaml'

In [41]:
import tensorflow
tensorflow.__version__

'2.13.0'

In [43]:
import tensorflow 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model



SyntaxError: invalid syntax (1022079985.py, line 5)

In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

class Evaluation:
    def __init__(self, config):
        self.config = config

    def evaluation(self):
        self.model = load_model(self.config.path_of_model)
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        print(scores)

# Example usage
try:
    # Assuming ConfigurationManager is properly defined and returns config
    config = ConfigurationManager().get_evaluation_config()
    evaluation = Evaluation(config)
    evaluation.evaluation()

except Exception as e:
    print(e)


[2024-06-28 12:33:40,252: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-28 12:33:40,256: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-28 12:33:40,259: INFO: common: created directory at: artifacts]
'Evaluation' object has no attribute 'valid_generator'


In [22]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import mlflow
from urllib.parse import urlparse
from pathlib import Path
import json

# Ensure correct TensorFlow version
print(tf.__version__)
print(keras.__version__)

class EvaluationConfig:
    def __init__(self, params_image_size, params_batch_size, training_data, path_of_model, mlflow_uri, all_params):
        self.params_image_size = params_image_size
        self.params_batch_size = params_batch_size
        self.training_data = training_data
        self.path_of_model = path_of_model
        self.mlflow_uri = mlflow_uri
        self.all_params = all_params

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

def save_json(path: Path, data: dict):
    with open(path, 'w') as f:
        json.dump(data, f)

# Example usage
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e


2.13.0
2.13.1
[2024-06-28 12:41:52,712: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-28 12:41:52,715: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-28 12:41:52,717: INFO: common: created directory at: artifacts]
Found 129 images belonging to 2 classes.
9/9 [==============================] - 27s 3s/step - loss: 0.0866 - accuracy: 0.9690


2024/06/28 12:42:22 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/fg/ghp__2t92s17zm8yvjw3nphh0000gn/T/tmpfc_werhl/model/data/model/assets
[2024-06-28 12:42:23,062: INFO: builder_impl: Assets written to: /var/folders/fg/ghp__2t92s17zm8yvjw3nphh0000gn/T/tmpfc_werhl/model/data/model/assets]


AttributeError: module 'tensorflow.keras' has no attribute '__version__'

In [48]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import mlflow
from urllib.parse import urlparse
from pathlib import Path
import json

class Evaluation:
    def __init__(self, config):
        self.config = config
        print("TensorFlow version:", tf.__version__)
        # print("Keras version:", keras.__version__)

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path):
        return load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

# Example usage
try:
    # Assuming ConfigurationManager is properly defined and returns config
    config = ConfigurationManager().get_evaluation_config()
    evaluation = Evaluation(config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e


[2024-06-28 13:23:30,012: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-28 13:23:30,015: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-28 13:23:30,017: INFO: common: created directory at: artifacts]
TensorFlow version: 2.13.0
Found 129 images belonging to 2 classes.
9/9 [==============================] - 26s 3s/step - loss: 9.4563 - accuracy: 0.4651
[2024-06-28 13:23:56,678: INFO: common: json file saved at: scores.json]


2024/06/28 13:23:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/fg/ghp__2t92s17zm8yvjw3nphh0000gn/T/tmplym6nejb/model/data/model/assets
[2024-06-28 13:23:58,741: INFO: builder_impl: Assets written to: /var/folders/fg/ghp__2t92s17zm8yvjw3nphh0000gn/T/tmplym6nejb/model/data/model/assets]


AttributeError: module 'tensorflow.keras' has no attribute '__version__'